## Run All ipynb Ini

In [43]:
%pip install pandas numpy imbalanced-learn scikit-learn xgboost hyperopt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn import model_selection
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


In [45]:
dataFramePenyakitJantung = pd.read_csv("./dataset/heart_disease_uci.csv")

In [46]:
# DROP KOLOM TIDAK PERLU
dataFramePenyakitJantung = dataFramePenyakitJantung.drop(columns=["id", "dataset"])
dataFramePenyakitJantung

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,Female,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,62,Male,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,55,Male,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,58,Male,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [47]:
# duplikat 2
print(dataFramePenyakitJantung.duplicated().sum())
dataFramePenyakitJantung = dataFramePenyakitJantung.drop_duplicates()
print(dataFramePenyakitJantung.duplicated().sum())


2
0


In [48]:

print(dataFramePenyakitJantung.isnull().sum())
print("Persentasi Nilai Hilang:\n", dataFramePenyakitJantung.isnull().sum() / len(dataFramePenyakitJantung) * 100)

# Melakukan imputasi ke missing value
numerical_cols = dataFramePenyakitJantung.select_dtypes(include=np.number).columns.tolist()
categorical_cols = dataFramePenyakitJantung.select_dtypes(include='object').columns.tolist()

imputer_numerical = SimpleImputer(strategy='mean')

imputer_numerical.fit(dataFramePenyakitJantung[numerical_cols])
df_imputed_num = imputer_numerical.transform(dataFramePenyakitJantung[numerical_cols])


imputer_categorical = SimpleImputer(strategy='most_frequent')

imputer_categorical.fit(dataFramePenyakitJantung[categorical_cols])

df_imputed_cat = imputer_categorical.transform(dataFramePenyakitJantung[categorical_cols])

imputed_df = pd.DataFrame(df_imputed_num, columns=numerical_cols, index=dataFramePenyakitJantung.index)
imputed_df[categorical_cols] = df_imputed_cat


print("Setelah di imputasi")
print(imputed_df.isnull().sum())

imputed_df


age           0
sex           0
cp            0
trestbps     59
chol         29
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       307
ca          609
thal        484
num           0
dtype: int64
Persentasi Nilai Hilang:
 age          0.000000
sex          0.000000
cp           0.000000
trestbps     6.427015
chol         3.159041
fbs          9.803922
restecg      0.217865
thalch       5.991285
exang        5.991285
oldpeak      6.753813
slope       33.442266
ca          66.339869
thal        52.723312
num          0.000000
dtype: float64
Setelah di imputasi
age         0
trestbps    0
chol        0
thalch      0
oldpeak     0
ca          0
num         0
sex         0
cp          0
fbs         0
restecg     0
exang       0
slope       0
thal        0
dtype: int64


,age,trestbps,chol,thalch,oldpeak,ca,num,sex,cp,fbs,restecg,exang,slope,thal
0,63.0,145.000000,233.0,150.000000,2.300000,0.000000,0.0,Male,typical angina,True,lv hypertrophy,False,downsloping,fixed defect
1,67.0,160.000000,286.0,108.000000,1.500000,3.000000,2.0,Male,asymptomatic,False,lv hypertrophy,True,flat,normal
2,67.0,120.000000,229.0,129.000000,2.600000,2.000000,1.0,Male,asymptomatic,False,lv hypertrophy,True,flat,reversable defect
3,37.0,130.000000,250.0,187.000000,3.500000,0.000000,0.0,Male,non-anginal,False,normal,False,downsloping,normal
4,41.0,130.000000,204.0,172.000000,1.400000,0.000000,0.0,Female,atypical angina,False,lv hypertrophy,False,upsloping,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54.0,127.000000,333.0,154.000000,0.000000,0.676375,1.0,Female,asymptomatic,True,st-t abnormality,False,flat,normal
916,62.0,132.137369,139.0,137.542294,0.880841,0.676375,0.0,Male,typical angina,False,st-t abnormality,False,flat,normal
917,55.0,122.000000,223.0,100.000000,0.000000,0.676375,2.0,Male,asymptomatic,True,st-t abnormality,False,flat,fixed defect
918,58.0,132.137369,385.0,137.542294,0.880841,0.676375,0.0,Male,asymptomatic,True,lv hypertrophy,False,flat,normal


In [49]:
print(categorical_cols)

['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']


In [50]:
# Mengubah data kategorikal menjadi numeric menggunakan One-Hot Encoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

encoder.fit(df_imputed_cat)

df_encoded_cat = encoder.transform(df_imputed_cat)


encoded_feature_names = encoder.get_feature_names_out(categorical_cols)
categorical_df = pd.DataFrame(df_encoded_cat, columns=encoded_feature_names, index=dataFramePenyakitJantung.index)
num_df = pd.DataFrame(df_imputed_num, columns=numerical_cols, index=dataFramePenyakitJantung.index)

df_processed = pd.concat([num_df, categorical_df], axis=1)

df_processed.to_csv("./data_after_preprocess.csv")

df_processed.head()

,age,trestbps,chol,thalch,oldpeak,ca,num,sex_Female,sex_Male,cp_asymptomatic,...,restecg_normal,restecg_st-t abnormality,exang_False,exang_True,slope_downsloping,slope_flat,slope_upsloping,thal_fixed defect,thal_normal,thal_reversable defect
0,63.0,145.0,233.0,150.0,2.3,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,67.0,160.0,286.0,108.0,1.5,3.0,2.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,67.0,120.0,229.0,129.0,2.6,2.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,37.0,130.0,250.0,187.0,3.5,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,41.0,130.0,204.0,172.0,1.4,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [51]:
# Merubah dataset dari multi class ke binary
map_to_binary = {
    0.0: 0,
    1.0: 1,
    2.0: 1,
    3.0: 1,
    4.0: 1
}

df_processed["num"] = df_processed["num"].map(map_to_binary)

df_processed

,age,trestbps,chol,thalch,oldpeak,ca,num,sex_Female,sex_Male,cp_asymptomatic,...,restecg_normal,restecg_st-t abnormality,exang_False,exang_True,slope_downsloping,slope_flat,slope_upsloping,thal_fixed defect,thal_normal,thal_reversable defect
0,63.0,145.000000,233.0,150.000000,2.300000,0.000000,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,67.0,160.000000,286.0,108.000000,1.500000,3.000000,1,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,67.0,120.000000,229.0,129.000000,2.600000,2.000000,1,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,37.0,130.000000,250.0,187.000000,3.500000,0.000000,0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,41.0,130.000000,204.0,172.000000,1.400000,0.000000,0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54.0,127.000000,333.0,154.000000,0.000000,0.676375,1,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
916,62.0,132.137369,139.0,137.542294,0.880841,0.676375,0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
917,55.0,122.000000,223.0,100.000000,0.000000,0.676375,1,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
918,58.0,132.137369,385.0,137.542294,0.880841,0.676375,0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [52]:
X = df_processed.drop(columns="num")
y = df_processed['num']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)
y

0      0
1      1
2      1
3      0
4      0
      ..
915    1
916    0
917    1
918    0
919    1
Name: num, Length: 918, dtype: int64

In [53]:
# K = 5, untuk menghasilkan akurasi yang lebih baik
smote = SMOTE(random_state=42, k_neighbors=5)

X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print(f"Data train Sebelum di Oversampling: {X_train.shape[0]} baris")
print(f"Data train Sesudah di Oversampling: {X_resampled.shape[0]} baris")

print()
print("Jumlah Class 0 dan 1, sebelum di oversampling")
print(y_train.value_counts())
print()
print("Jumlah Class 0 dan 1, setelah di oversampling")
print(y_resampled.value_counts())

Data train Sebelum di Oversampling: 734 baris
Data train Sesudah di Oversampling: 794 baris

Jumlah Class 0 dan 1, sebelum di oversampling
num
1    397
0    337
Name: count, dtype: int64

Jumlah Class 0 dan 1, setelah di oversampling
num
1    397
0    397
Name: count, dtype: int64


In [54]:
search_space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1.0)),
    'max_depth': hp.quniform('max_depth', 3, 13, 1),
    'n_estimators': hp.quniform('n_estimators', 500, 1000, 1),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'gamma': hp.uniform('gamma', 0.5, 5.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    # Anda juga bisa menambahkan parameter lain di sini, contoh:
    # 'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
}


# 2. Definisikan fungsi objektif yang akan diminimalkan
# Hyperopt akan memanggil fungsi ini berulang kali dengan parameter yang berbeda.
def objective(params):
    """
    Fungsi objektif untuk hyperopt.
    Menerima dictionary parameter, melatih model, dan mengembalikan loss.
    """
    # hp.quniform menghasilkan float, jadi kita perlu mengubahnya menjadi integer
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])

    # Inisialisasi model dengan parameter yang diberikan
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        random_state=42,
        n_jobs=-1,
        **params
    )

    # Lakukan cross-validation untuk mengevaluasi parameter
    # Kita menggunakan .mean() untuk mendapatkan skor rata-rata
    accuracy = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()

    # Hyperopt bekerja untuk MINIMASI, jadi kita perlu mengembalikan nilai negatif dari akurasi
    # Semakin tinggi akurasi, semakin rendah (lebih negatif) loss-nya.
    loss = -accuracy

    # Mengembalikan dictionary yang dibutuhkan oleh hyperopt
    return {'loss': loss, 'status': STATUS_OK, 'params': params}


# 3. Jalankan optimisasi dengan fmin
# 'fmin' akan mencari parameter terbaik yang meminimalkan fungsi objektif.
print("Memulai Optimisasi Bayesian dengan Hyperopt...")

# Trials object untuk menyimpan histori dari setiap iterasi
trials = Trials()

# Jalankan optimisasi
best_params = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,  # Tree-structured Parzen Estimator, algoritma optimisasi
    max_evals=50,      # Jumlah total kombinasi parameter untuk dicoba
    trials=trials,
    rstate=np.random.default_rng(42) # Untuk reproduktifitas
)


# 4. Tampilkan hasil terbaik
# fmin mengembalikan parameter terbaik, kita bisa mencetaknya atau mengambil dari object trials
print("\nOptimisasi Selesai.")
best_run = trials.best_trial['result']['params']
print(f"Parameter terbaik ditemukan: {best_run}")
print(f"Skor cross-validation terbaik (Akurasi): {-trials.best_trial['result']['loss']:.4f}")


# 5. Latih model final dengan parameter terbaik
print("\nMelatih model final dengan parameter terbaik...")
# Pastikan tipe data integer benar sebelum melatih model final
best_run['max_depth'] = int(best_run['max_depth'])
best_run['n_estimators'] = int(best_run['n_estimators'])
best_run['min_child_weight'] = int(best_run['min_child_weight'])

best_xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1,
    **best_run
)

best_xgb_model.fit(X_train, y_train)
print("Model final siap digunakan.")


Memulai Optimisasi Bayesian dengan Hyperopt...
100%|██████████| 50/50 [00:58<00:00,  1.16s/trial, best loss: -0.8324387289162241]

Optimisasi Selesai.
Parameter terbaik ditemukan: {'colsample_bytree': 0.6312082363473505, 'gamma': 1.5553948791373746, 'learning_rate': 0.020241183097896465, 'max_depth': 4, 'min_child_weight': 10, 'n_estimators': 675, 'reg_lambda': 0.10308206613656157, 'subsample': 0.7299796125452439}
Skor cross-validation terbaik (Akurasi): 0.8324

Melatih model final dengan parameter terbaik...


c:\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:16:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model final siap digunakan.


In [55]:
y_pred_best = best_xgb_model.predict(X_test)

X_test

,age,trestbps,chol,thalch,oldpeak,ca,sex_Female,sex_Male,cp_asymptomatic,cp_atypical angina,...,restecg_normal,restecg_st-t abnormality,exang_False,exang_True,slope_downsloping,slope_flat,slope_upsloping,thal_fixed defect,thal_normal,thal_reversable defect
669,59.0,125.000000,0.0,175.000000,2.600000,0.676375,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
30,69.0,140.000000,239.0,151.000000,1.800000,2.000000,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
377,45.0,140.000000,224.0,122.000000,0.000000,0.676375,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
536,48.0,122.000000,275.0,150.000000,2.000000,0.676375,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
808,62.0,158.000000,210.0,112.000000,3.000000,0.676375,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,38.0,120.000000,231.0,182.000000,3.800000,0.000000,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
746,51.0,132.137369,0.0,137.542294,0.880841,0.676375,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
585,47.0,120.000000,205.0,98.000000,2.000000,0.676375,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
879,56.0,132.137369,282.0,137.542294,0.880841,0.676375,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [56]:
print("Best Model XGB")
accuracy = accuracy_score(y_test, y_pred_best)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

Best Model XGB
Accuracy: 0.8207

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.78      0.78        73
           1       0.85      0.85      0.85       111

    accuracy                           0.82       184
   macro avg       0.81      0.81      0.81       184
weighted avg       0.82      0.82      0.82       184



In [57]:
nama_model = "klasifikasi_penyakit_jantung_xgboost.pkl"

with open(nama_model, 'wb') as file:
    pickle.dump(best_xgb_model, file)